## MODEL CLASS
_Represent a CNN model_

In [ ]:
import sys
import os
import pickle
import numpy as np
from sklearn import model_selection,preprocessing
from keras.utils import np_utils
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Dense, Flatten
from keras.optimizers import SGD, Adam
from keras.regularizers import l1,l2, l1_l2
# import python library
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))

from dataset import Dataset
from database import Database
from sample import Sample


class Model(Database):
    
    __X_train, __X_test, __y_train, __y_test = None, None, None, None
    __data_shape = None
    __model = None
    
    # Initializer
    def __init__(self, dataset_location, database_location, site):
#         super(Database, self).__init__(dataset_location, database_location, site) 
        super().__init__(dataset_location, database_location, site)       
    
    def get_X_train(self):
        if self.__X_train is None:
            self.train_test_split()
        return self.__X_train
    
    def get_X_test(self):
        if self.__X_test is None:
            self.train_test_split()
        return self.__X_test
    
    def get_y_train(self):
        if self.__y_train is None:
            self.train_test_split()
        return self.__y_train
    
    def get_y_test(self):
        if self.__y_test is None:
            self.train_test_split()
        return self.__y_test
    
    def get_shape(self):
        return self.__data_shape
    
    def train_test_split(self, test_size=0.33, random_state=42):
        X, y = self.load_cuts_db_mp()
        self.__X_train, self.__X_test, self.__y_train, self.__y_test =  model_selection.train_test_split(X, y,
                                                                     test_size=test_size, random_state=random_state)
        self.__data_shape = self.__X_train[-1].shape
        
    def scale_data(self):
        if self.__data_shape is None:
            self.train_test_split()
        self.__X_train = self.__X_train.reshape(len(self.__X_train), self.__data_shape[0]*self.__data_shape[1])
        self.__X_test = self.__X_test.reshape(len(self.__X_test), self.__data_shape[0]*self.__data_shape[1])
        scaler = preprocessing.StandardScaler().fit(self.__X_train)
        self.__X_train = scaler.transform(self.__X_train)
        self.__X_test = scaler.transform(self.__X_test)
        # reshape for conv net
        self.__X_train = self.__X_train.reshape(len(self.__X_train), 1, self.__data_shape[0], self.__data_shape[1])
        self.__y_train = np_utils.to_categorical(self.__y_train)
        self.__X_test = self.__X_test.reshape(len(self.__X_test), 1, self.__data_shape[0], self.__data_shape[1])
        self.__y_test = np_utils.to_categorical(self.__y_test)
    
    def plot_samples(self, train=True):
        size = 10
        sample_per_row = 5
        plt.figure(figsize=(size*2,size))
        for i in range(0,(sample_per_row**2)*2,2):
            k = np.random.randint(len(self.__X_train)) if train else np.random.randint(len(self.__X_test)) 
            plt.subplot(sample_per_row,sample_per_row*2,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.pcolormesh(self.__X_train[k][0], cmap='jet') if train else plt.pcolormesh(self.__X_test[k][0], cmap='jet')
            plt.xlabel(np.bool_(self.__y_train[k][1])) if train else plt.xlabel(np.bool_(self.__y_test[k][1]))   
            plt.subplot(sample_per_row,sample_per_row*2,i+2)
            plt.yticks([])
            plt.grid(False)
            plt.hist(self.__X_train[k][0].flatten(), bins='auto') if train else plt.hist(self.__X_test[k][0].flatten(), bins='auto')
        plt.show()
        
    def create_model(self):
        # create the model
        model = Sequential()
        model.add(Conv2D(2,(32,32), input_shape=(1,self.__data_shape[0], self.__data_shape[1]),
                        padding='same', activation='relu',data_format='channels_first'))
        model.add(Dropout(0.2))
        model.add(Conv2D(4, (16,16), activation='relu', padding='same',data_format='channels_first'))
        model.add(Dropout(0.4))
        model.add(Conv2D(8, (8,8), activation='relu', padding='same',data_format='channels_first'))
        model.add(MaxPooling2D(pool_size=(2,2),data_format='channels_first'))
        model.add(Conv2D(2, (64,64), activation='relu', padding='same',data_format='channels_first'))
#         model.add(MaxPooling2D(pool_size=(2,2),data_format='channels_first'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.6))
        model.add(Dense(2, activation='softmax'))
        self.__model = model
        
    def compile_model(self,optimizer, loss='categorical_crossentropy', metrics=['accuracy'], verbose=True):
        self.__model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        print(self.__model.summary()) if verbose else None
        
    def train_model(self, validation_split, epochs, batch_size, metrics=True):
        history = self.__model.fit(self.__X_train, self.__y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size)
        if metrics:
            plt.figure()
            plt.plot(history.history['acc'])
            plt.plot(history.history['val_acc'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper left')
            plt.show()
            # summarize history for loss
            plt.figure()
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper left')
            plt.show()
        return history
    
    def get_conv_layers():
        layers = []
        # summarize filter shapes
        for layer in self.__model.layers:
            # check for convolutional layer
            if 'conv' not in layer.name:
                continue
            # get filter weights
            layers.append(layer)
        return layers

    def visualization(summary=False, plot=False, conv_layer_number=0):
        '''Visualize the CNN model'''
        if summary:
            print(self.__model.summary())
        if plot:
            plot_model(self.__model, show_shapes=True, show_layer_names=True, to_file='model.png')
            Image(retina=True, filename='model.png')
        # get filers and biases
        filters, biases = self.get_conv_layers()[conv_layer_number].get_weights()
        # scale filters
        f_min, f_max = filters.min(), filters.max()
        filters = (filters-f_min)/(f_max-f_min)
        # get number of filters
        n_filters, ix = filters.shape[-1], 1

        for i in range(n_filters):
            # get the filters
            f = filters[:,:,:,i]
            # plot each channel seperately
            for j in range(1):
                # specify subplot and turn of axis
                ax = plt.subplot(1,n_filters, ix)
                ax.set_xticks([])
                ax.set_yticks([])
                # plot filter channel in grayscale
                plt.imshow(f[:, :, j], cmap='jet')
                ix += 1
        plt.show()

    def visualization_fm(conv_layer_number=0):
        data = self.__X_train[np.random.randint(len(self.__X_train))]
        feature_maps = self.__model.predict([data])
        for i in range(n_filters):
            # get the filters
            f = filters[:,:,:,i]
            # plot each channel seperately
            for j in range(1):
                # specify subplot and turn of axis
                ax = plt.subplot(1,n_filters, ix)
                ax.set_xticks([])
                ax.set_yticks([])
                # plot filter channel in grayscale
                plt.pcolormesh(f[:, :, j], cmap='jet')
                ix += 1
        plt.show()